In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import HTML
def css_styling():
    styles = open("../input/titlestyle/style2.css", "r").read()
    return HTML(styles)
css_styling()

<div class="heading">
   <h1><span style="color: white">Intro</span></h1>
</div>
<div class="content">

<u>📔 Public notebooks - 130+</u><br>

<u>🥇 Gold medals - 6+</u><br>

<u>🥈 Silver medals - 18+</u><br>

<u>🥉 Bronze medals - 41+</u><br>


Wordcloud made of notebook titles:
<img src="https://i.imgur.com/hCvj3cP.png" alt="img1"/>
</div>

<div style = "font-family: Arial;font-size:1.6em;color: #0a6121;background: #ace6bc;padding:5px;border-style: solid;border-color:#0a6121;">
<b>Summa summarum:</b> LightGBM, ANN, lag target features
</div>

<div class="heading">
   <h1><span style="color: white">Goal</span></h1>
</div>
<div class='content'>
    <h3><b>⚾ Forecast supporters digital engagement (time-series forecasting) using data such as player performance, social media and team factors.</b></h3>
    <h3><b>⚾ The supporters engagement is provided through 4 different time-series. The real meaning of the series is not explained but it may be related to likes, retweets and replies on Twitter or something similar.</b></h3>
</div>

In [ ]:
data_dir = '/kaggle/input/mlb-player-digital-engagement-forecasting'

df = pd.read_csv(f"{data_dir}/train.csv").dropna(axis=1,how='all')
daily_data_nested_df_names = df.drop('date', axis = 1).columns.values.tolist()

df_name = 'nextDayPlayerEngagement'
date_nested_table = df[['date', df_name]]

date_nested_table = (date_nested_table[
  ~pd.isna(date_nested_table[df_name])].reset_index(drop = True))

daily_dfs_collection = []

for date_index, date_row in date_nested_table.iterrows():
    daily_df = pd.read_json(date_row[df_name])

    daily_df['dailyDataDate'] = date_row['date']

    daily_dfs_collection = daily_dfs_collection + [daily_df]

unnested_table = (pd.concat(daily_dfs_collection,ignore_index = True).
                  set_index('dailyDataDate').reset_index())

unnested_table.to_pickle(f"train_{df_name}.pickle")

del(date_nested_table, daily_dfs_collection, unnested_table)
        
train_target = pd.read_pickle('train_nextDayPlayerEngagement.pickle')
train_target['engagementMetricsDate'] = pd.to_datetime(train_target['engagementMetricsDate'])
train_target['dailyDataDate'] = train_target['dailyDataDate'].astype(str)
train_target['dailyDataDate'] = pd.to_datetime(train_target['dailyDataDate'], format="%Y%m%d")

fig, axes = plt.subplots(4, 1, figsize=(12,15))

for i, ax in enumerate(axes):
    train_target.groupby('engagementMetricsDate').mean()[f'target{i+1}'].plot(ax=ax)
    ax.set_title(f'mean target{i+1}')

<div class="heading">
   <h1><span style="color: white">Ideas</span></h1>
</div>
<div class='content'>
This topic won't go into the domain range of this competition but rather to summarize the technical ideas shared in the notebooks. Nevertheless, if you are interested in <b>applications of DS in baseball</b>, feel free to check this discussion <a href="https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/discussion/245334">https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/discussion/245334</a> as well as other baseball data sets <a href="https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/discussion/245328">https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/discussion/245328</a><br><br>

Cross-validation (CV) is a technique used in machine learning to determine how well the forecasts are generalizing. The goal of this process is to estimate the error rate of the model - how far off are our predictions from reality. It does this by splitting up historical data into different groups and using one group as a test set and then comparing it against an independent set of data that was not used for training.<br><br>

In opposite to k-fold CV, where the data set is divided into k-folds and each fold is treated as a test set in one iteration, <b>CV for time-series</b> has a slightly different logic. The main reason is that time-series are time depending data and it wouldn't be accurate to use data from the future as a training set to predict past data. While splitting the data, we have to follow the time order to simulate real-life data flow. This can be done using either a fixed or expanding training set.<br><br>

<img src="https://miro.medium.com/max/1204/1*qvdnPF8ETV9mFdMT0Y_BBA.png" alt="img1"/>
<center><i>Time-series CV, Source: <a href="https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4">https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4</a></i></center>
<a href=""></a>

The neural network is a computational system modeled after the human brain, which uses a series of interconnected layers that loosely represent neurons in the brain. They work by trying to identify patterns in data, and then use that knowledge to make predictions about new data. The example of <b>ANN implemented in Keras</b> that predicts all four targets at once is presented here <a href="https://www.kaggle.com/ulrich07/mlb-ann-with-lags-tf-keras">https://www.kaggle.com/ulrich07/mlb-ann-with-lags-tf-keras</a> <b>ANN implemented in PyTorch</b> is described here <a href="https://www.kaggle.com/nicohrubec/mlb-pytorch-dnn-loop-feature-engineering">https://www.kaggle.com/nicohrubec/mlb-pytorch-dnn-loop-feature-engineering</a><br><br>

LightGBM is a predictive analytics library for Python that can be used with both CPU and GPU. It is a scalable, unified model predictive framework that supports tree-based models. In comparison to XGBoost, the Light GBM is almost 7 times faster which is a much better approach when dealing with large datasets. This turns out to be a huge advantage when you are working on large datasets in limited-time competitions.<br><br>

A Cat Boosting regressor is a regression algorithm that improves model performance by sequentially adding weaker models. Generally, it can be used for both classification and regression problems. The initial model is trained using all of the data and then subsequent models are trained using subsets of the data that are too noisy to predict well, but contain not enough data to train a good model without overtraining. Example of <b>LightGBM plus CatBoosting</b> can be found here <a href="https://www.kaggle.com/lhagiimn/lightgbm-catboost-ann-2505f2">https://www.kaggle.com/lhagiimn/lightgbm-catboost-ann-2505f2</a>
Similarly, <b>LightGBM + ANN in Keras</b> is here <a href="https://www.kaggle.com/mlconsult/1-35-lightgbm-ann">https://www.kaggle.com/mlconsult/1-35-lightgbm-ann</a>
    
<img src="https://image.slidesharecdn.com/xgboostandlightgbm-180201121028/95/xgboost-lightgbm-21-638.jpg?cb=1517487076" alt="img3"/>
<center><i>Difference between XGBoost and LightGBM, <a href="https://www.slideshare.net/GabrielCyprianoSaca/xgboost-lightgbm">https://www.slideshare.net/GabrielCyprianoSaca/xgboost-lightgbm</a></i></center><br>
    
Some classic time-series forecasting methods, such as <b>SARIMAX, VAR, and HWES</b> can recognize autoregression, moving average or seasonal parts of time series. In this competition, some of the econometrics methods have been tested here <a href="https://www.kaggle.com/adarshsng/time-series-forecasting-case-study">https://www.kaggle.com/adarshsng/time-series-forecasting-case-study</a> but you can find a more <b>detailed tutorial about economentrics and time-series</b> here <a href="https://www.kaggle.com/eneszvo/time-series-forecasting-p1-es-arima-var">https://www.kaggle.com/eneszvo/time-series-forecasting-p1-es-arima-var</a>.<br><br>
    
💡 Even if lag targets are not provided in the private test, you can save your own predictions and use them as lag targets. This <b>approach with different lags</b> seemed to work for the current leader in the competition <a href="https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/discussion/250945#1383850">https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/discussion/250945#1383850</a>

</div>
